#### 1.3. 토크나이저 불러오기

In [ ]:
# 파일 업로드 (Google Colab의 경우)
from google.colab import files
uploaded = files.upload()

# 파일 불러오기
with open('X_train_tokenized.pkl', 'rb') as f:
    X_train_tokenized = pickle.load(f)

with open('X_test_tokenized.pkl', 'rb') as f:
    X_test_tokenized = pickle.load(f)

with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)


### 2.1 Attention + LSTM 모델

Attention 매커니즘

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Attention 가중치를 위한 weight 생성
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        # Attention score 계산
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])


LSTM 구현

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input

VOCAB_SIZE = len(tokenizer.word_index) + 1  # 단어 사전의 크기
EMBEDDING_DIM = 128  # 임베딩 벡터의 차원
MAX_LENGTH = 100  # 입력 시퀀스의 최대 길이 (앞서 패딩 처리할 때 설정한 값)

# 모델 구성
input_layer = Input(shape=(MAX_LENGTH,))
embedding_layer = Embedding(VOCAB_SIZE, EMBEDDING_DIM)(input_layer)
lstm_layer = LSTM(128, return_sequences=True)(embedding_layer)
attention_layer = AttentionLayer()(lstm_layer)
output_layer = Dense(1, activation="sigmoid")(attention_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()


### 2.2 KoBERT 모델

KoBERT를 활용하려면 Huggingface의 Transformers 라이브러리가 필요합니다. KoBERT 모델을 불러와서 사용할 수 있습니다.

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam

# KoBERT 토큰화기와 모델 불러오기
tokenizer_kobert = BertTokenizer.from_pretrained('monologg/kobert')
model_kobert = TFBertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=2)

# 모델 구성 및 컴파일
model_kobert.compile(optimizer=Adam(learning_rate=5e-5), loss='binary_crossentropy', metrics=['accuracy'])
model_kobert.summary()

이제 두 모델이 준비되었습니다. 각각의 모델로 텍스트 데이터를 학습시키면 스팸 여부를 분류할 수 있게 됩니다.

### 3. 학습 및 검증

#### 3.1 Attention + LSTM 모델

1. **모델에 Dropout 추가**:
Dropout을 LSTM 레이어와 출력 레이어 사이에 추가합니다.

In [ ]:
from tensorflow.keras.layers import Dropout

input_layer = Input(shape=(MAX_LENGTH,))
embedding_layer = Embedding(VOCAB_SIZE, EMBEDDING_DIM)(input_layer)
lstm_layer = LSTM(128, return_sequences=True)(embedding_layer)
attention_layer = AttentionLayer()(lstm_layer)
dropout_layer = Dropout(0.5)(attention_layer)  # Dropout 추가
output_layer = Dense(1, activation="sigmoid")(dropout_layer)

model_attention_lstm = Model(inputs=input_layer, outputs=output_layer)
model_attention_lstm.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

2. **Early Stopping 추가 및 모델 학습**:

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

history_attention_lstm = model_attention_lstm.fit(
    X_train_padded, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test_padded, y_test),
    callbacks=[early_stopping]
)

#### 3.2 KoBERT 모델

1. **Early Stopping 추가 및 모델 학습**:

KoBERT를 사용할 때, 입력 데이터는 토큰화 후 패딩 처리된 것이 아닌 원래의 텍스트 데이터를 사용해야 합니다.


In [ ]:
train_encodings = tokenizer_kobert(list(X_train), truncation=True, padding=True, max_length=MAX_LENGTH)
test_encodings = tokenizer_kobert(list(X_test), truncation=True, padding=True, max_length=MAX_LENGTH)

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

history_kobert = model_kobert.fit(
    train_encodings, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(test_encodings, y_test),
    callbacks=[early_stopping]

위의 코드를 통해 각 모델을 학습하고 검증 데이터를 사용하여 성능을 평가할 수 있습니다. Early Stopping을 사용하여 성능 향상이 없을 때 학습을 조기 종료하는 기법을 적용하였습니다.

### 4. 앙상블

앙상블에서 가장 간단하면서도 효과적인 방법은 소프트 투표(확률 평균)입니다. 각 모델의 예측 확률을 평균내어 최종 예측 확률을 계산하고, 그 확률을 기반으로 분류를 수행합니다.


In [ ]:
# 각 모델의 예측 확률 계산
prob_attention_lstm = model_attention_lstm.predict(X_test_padded)
prob_kobert = model_kobert.predict(test_encodings)[0]  # transformers 모델의 출력은 (logit, ...) 형태이므로 [0] 선택

# 확률을 평균내어 최종 예측 확률 계산
final_prob = (prob_attention_lstm + prob_kobert) / 2

# 확률을 기반으로 최종 분류 수행
final_predictions = [1 if prob > 0.5 else 0 for prob in final_prob]

### 5. 최종 평가

최종적으로 앙상블 모델의 성능을 평가합니다. 분류 문제에서 일반적으로 사용되는 지표는 정확도, 정밀도, 재현율, F1-점수 등입니다.


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("정확도:", accuracy_score(y_test, final_predictions))
print("정밀도:", precision_score(y_test, final_predictions))
print("재현율:", recall_score(y_test, final_predictions))
print("F1-점수:", f1_score(y_test, final_predictions))


위의 코드를 통해 앙상블 모델을 구성하고 최종 성능을 평가할 수 있습니다.

### 6. 하이퍼파라미터 튜닝

하이퍼파라미터 튜닝은 모델의 성능을 향상시키는 중요한 단계입니다.

#### Attention + LSTM 모델:

1. **임베딩 차원(EMBEDDING_DIM)**: 단어를 몇 차원의 벡터로 표현할지 결정합니다.
2. **LSTM 유닛의 수**: LSTM 레이어의 유닛 수를 조정합니다.
3. **Dropout 비율**: 오버피팅을 방지하기 위한 Dropout의 비율을 조정합니다.
4. **학습률(learning rate)**: 옵티마이저의 학습률을 조정합니다.
5. **배치 크기(batch size)**: 한 번에 학습하는 데이터의 수를 조정합니다.

#### KoBERT 모델:

1. **학습률(learning rate)**: 옵티마이저의 학습률을 조정합니다.
2. **배치 크기(batch size)**: 한 번에 학습하는 데이터의 수를 조정합니다.

### 2. 튜닝 방법:

#### Grid Search:
- 가능한 모든 조합의 하이퍼파라미터 값을 시도하여 최적의 값을 찾습니다.
- 시간이 오래 걸리는 단점이 있습니다.

#### Random Search:
- 주어진 하이퍼파라미터 값의 범위 내에서 무작위로 값을 선택하여 최적의 값을 찾습니다.
- Grid Search보다 빠르게 근사치를 찾을 수 있습니다.

### 조언:

1. **시간과 리소스**: Grid Search는 모든 조합을 시도하기 때문에 많은 시간과 리소스가 필요합니다. 이에 반해 Random Search는 주어진 시간 내에서 더 다양한 조합을 탐색할 수 있습니다. 따라서, 초기 단계에서는 Random Search를 사용하여 대략적인 최적의 하이퍼파라미터 범위를 찾은 후, 그 범위 내에서 Grid Search를 수행하는 것이 좋습니다.

2. **하이퍼파라미터 범위 설정**: 주요 튜닝 대상 하이퍼파라미터에 대해 넓은 범위를 설정하여 탐색을 시작하고, 점차 범위를 좁혀 나가는 것이 효과적입니다.

3. **Early Stopping**: 하이퍼파라미터 튜닝 과정에서도 Early Stopping을 활용하여 학습 시간을 단축시킬 수 있습니다.

4. **교차 검증(Cross Validation)**: 튜닝 과정에서 교차 검증을 활용하면 모델의 일반화 성능을 더 정확하게 평가할 수 있습니다. 하지만 시간이 더 많이 소요됩니다.

최적의 하이퍼파라미터를 찾는 과정은 많은 시간과 실험이 필요합니다. 따라서, 충분한 시간과 리소스를 확보하고 여러 번의 시도를 통해 최적의 값을 찾는 것이 중요합니다.